In [1]:
import pandas as pd

# Rutas de tus archivos CSV
ruta_resultados = "resultados_sonora.csv"
ruta_sitios = "sitios_sonora.csv"

# Cargar los archivos
df_resultados = pd.read_csv(ruta_resultados)
df_sitios = pd.read_csv(ruta_sitios)

# Hacer merge en base a la columna 'CLAVE SITIO'
df_merged = df_resultados.merge(df_sitios, on='CLAVE SITIO', how='left')

# Verifica las primeras filas
print(df_merged.head())

# Guarda si quieres trabajar con él desde otro notebook o proceso
df_merged.to_csv("datos_sonora_unificados.csv", index=False)


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\237221976.py:8: DtypeWarning: Columns (38,45,46,61,62,63,74,75,76,77,78,79,80,81,82,83,84,85,86,87,91,93,108,110,113,116,117,121,143,209,223,224,226,228,229,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resultados = pd.read_csv(ruta_resultados)


  CLAVE SITIO   CLAVE MONITOREO NOMBRE DEL SITIO_x TIPO CUERPO DE AGUA  \
0   OCNOR3984  OCNOR3984-251112             POZO 6         SUBTERRÁNEO   
1   OCNOR3984  OCNOR3984-100313             POZO 6         SUBTERRÁNEO   
2   OCNOR3984  OCNOR3984-020314             POZO 6         SUBTERRÁNEO   
3   OCNOR3984  OCNOR3984-220315             POZO 6         SUBTERRÁNEO   
4   OCNOR3984  OCNOR3984-150516             POZO 6         SUBTERRÁNEO   

  FECHA REALIZACIÓN   Año ALC_FEN ALC_TOT  CO3    HCO3  ...  CLAVE ACUÍFERO  \
0          15/11/12  2012     <25   152.6  0.0  152.60  ...          2629.0   
1          06/03/13  2013     <25  136.88  0.0  136.88  ...          2629.0   
2          05/03/14  2014     <25    91.8  0.0   91.80  ...          2629.0   
3          21/10/15  2015     <25  135.53  0.0  135.53  ...          2629.0   
4          18/05/16  2016     <25  140.58  0.0  140.58  ...          2629.0   

          ACUÍFERO ORGANISMO CUENCA  ESTADO    MUNICIPIO  \
0  RIO AGUA PRIETA  

In [2]:
columnas_agregadas = [col for col in df_sitios.columns if col in df_merged.columns and col not in df_resultados.columns]
print("Columnas nuevas agregadas del archivo de sitios:\n", columnas_agregadas)


Columnas nuevas agregadas del archivo de sitios:
 ['CUENCA', 'CLAVE ACUÍFERO', 'ACUÍFERO', 'ORGANISMO CUENCA', 'ESTADO', 'MUNICIPIO', 'CUERPO DE AGUA', 'TIPO DE CUERPO DE AGUA', 'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD']


In [3]:
df_merged['FECHA REALIZACIÓN'] = pd.to_datetime(df_merged['FECHA REALIZACIÓN'], dayfirst=True, errors='coerce')


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\646521178.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_merged['FECHA REALIZACIÓN'] = pd.to_datetime(df_merged['FECHA REALIZACIÓN'], dayfirst=True, errors='coerce')


In [4]:
print(df_merged['FECHA REALIZACIÓN'].min(), df_merged['FECHA REALIZACIÓN'].max())


2012-11-06 00:00:00 2022-10-10 00:00:00


In [5]:
df_merged['MES'] = df_merged['FECHA REALIZACIÓN'].dt.month
df_merged['AÑO'] = df_merged['FECHA REALIZACIÓN'].dt.year


In [6]:
import numpy as np

# 1. Convertir fecha a datetime
df_merged['FECHA REALIZACIÓN'] = pd.to_datetime(df_merged['FECHA REALIZACIÓN'], dayfirst=True, errors='coerce')
df_merged['MES'] = df_merged['FECHA REALIZACIÓN'].dt.month
df_merged['DIA'] = df_merged['FECHA REALIZACIÓN'].dt.day

# 2. Normalizar valores con '<', '>', 'ND', etc.
def limpiar_valor(valor):
    if isinstance(valor, str):
        if '<' in valor or '>' in valor:
            valor = valor.replace('<', '').replace('>', '')
        if valor.upper() in ['ND', 'NA', 'NR']:
            return np.nan
    try:
        return float(valor)
    except:
        return np.nan

# Aplicar limpieza solo a columnas numéricas esperadas
param_cols = df_merged.columns[6:230]  # Ajusta este rango si lo deseas
df_merged[param_cols] = df_merged[param_cols].applymap(limpiar_valor)

# 3. Eliminar columnas con más del 80% de valores nulos
limite_nulos = 0.8
df_merged = df_merged.loc[:, df_merged.isnull().mean() < limite_nulos]

# 4. Eliminar columnas constantes (solo un valor)
df_merged = df_merged.loc[:, df_merged.nunique() > 1]

# 5. Eliminar filas sin coordenadas geográficas si usarás mapas
df_merged = df_merged.dropna(subset=['LATITUD', 'LONGITUD'])

# Vista previa
print(df_merged.shape)
df_merged.head()


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\1789918265.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_merged[param_cols] = df_merged[param_cols].applymap(limpiar_valor)


(5900, 82)


,CLAVE SITIO,CLAVE MONITOREO,NOMBRE DEL SITIO_x,TIPO CUERPO DE AGUA,FECHA REALIZACIÓN,Año,ALC_TOT,CLOROF_A,COLI_FEC,COLI_TOT,...,ORGANISMO CUENCA,MUNICIPIO,CUERPO DE AGUA,TIPO DE CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD,MES,AÑO,DIA
0,OCNOR3984,OCNOR3984-251112,POZO 6,SUBTERRÁNEO,2012-11-15,2012,152.60,NaN,3.0,NaN,...,NOROESTE,AGUA PRIETA,ACUÍFERO RIO AGUA PRIETA,SUBTERRÁNEO,POZO,31.32013,-109.53205,11.0,2012.0,15.0
1,OCNOR3984,OCNOR3984-100313,POZO 6,SUBTERRÁNEO,2013-03-06,2013,136.88,NaN,1.0,NaN,...,NOROESTE,AGUA PRIETA,ACUÍFERO RIO AGUA PRIETA,SUBTERRÁNEO,POZO,31.32013,-109.53205,3.0,2013.0,6.0
2,OCNOR3984,OCNOR3984-020314,POZO 6,SUBTERRÁNEO,2014-03-05,2014,91.80,NaN,1.0,NaN,...,NOROESTE,AGUA PRIETA,ACUÍFERO RIO AGUA PRIETA,SUBTERRÁNEO,POZO,31.32013,-109.53205,3.0,2014.0,5.0
3,OCNOR3984,OCNOR3984-220315,POZO 6,SUBTERRÁNEO,2015-10-21,2015,135.53,NaN,1.0,NaN,...,NOROESTE,AGUA PRIETA,ACUÍFERO RIO AGUA PRIETA,SUBTERRÁNEO,POZO,31.32013,-109.53205,10.0,2015.0,21.0
4,OCNOR3984,OCNOR3984-150516,POZO 6,SUBTERRÁNEO,2016-05-18,2016,140.58,NaN,52.0,NaN,...,NOROESTE,AGUA PRIETA,ACUÍFERO RIO AGUA PRIETA,SUBTERRÁNEO,POZO,31.32013,-109.53205,5.0,2016.0,18.0


In [7]:
# Porcentaje de valores nulos por columna
nulos_porcentaje = df_merged.isnull().mean().sort_values(ascending=False)
print(nulos_porcentaje.head(20))  # Las 20 columnas más incompletas


PO4_TOT              0.796271
ALC_TOT              0.794068
FLUORUROS_TOT        0.792373
ACUÍFERO             0.788136
CLAVE ACUÍFERO       0.788136
FE_TOT               0.784237
MN_TOT               0.783729
ZN_TOT               0.775593
TOX_FIS_FON_30_UT    0.755085
TOX_FIS_FON_15_UT    0.755085
ENTEROC_FEC          0.720847
SALINIDAD_SUP        0.720000
PROFUNDIDAD          0.719153
TRANSPARENCIA        0.700169
OD_%_SUP             0.675085
TEMP_AGUA_SUP        0.674237
OD_mg/L_SUP          0.673898
CLOROF_A             0.670169
CONDUC_CAMPO_SUP     0.659831
pH_CAMPO_SUP         0.659831
dtype: float64


In [8]:
# Eliminar columnas constantes
df_merged = df_merged.loc[:, df_merged.nunique() > 1]


In [9]:
# Checar columnas no numéricas entre los parámetros
tipos = df_merged.dtypes
parametros_no_numericos = tipos[(tipos != 'float64') & (tipos != 'int64')]
print(parametros_no_numericos)


CLAVE SITIO                       object
CLAVE MONITOREO                   object
NOMBRE DEL SITIO_x                object
TIPO CUERPO DE AGUA               object
FECHA REALIZACIÓN         datetime64[ns]
NOMBRE DEL SITIO_y                object
CUENCA                            object
ACUÍFERO                          object
ORGANISMO CUENCA                  object
MUNICIPIO                         object
CUERPO DE AGUA                    object
TIPO DE CUERPO DE AGUA            object
SUBTIPO CUERPO AGUA               object
dtype: object


In [13]:
# Cargar simbología
simbologia_df = pd.read_csv("simbologia_sonora.csv")

# Verificar las columnas presentes en df_merged
parametros_disponibles = set(df_merged.columns)

# Filtrar simbología para los parámetros que sí están en los resultados
simbologia_filtrada = simbologia_df[simbologia_df['CLAVE PARÁMETRO'].isin(parametros_disponibles)]

# Mostrar simbología filtrada para definir subconjuntos
# Mostrar los primeros parámetros con descripción disponibles
print(simbologia_filtrada.head(20))


      CLAVE PARÁMETRO                              NOMBRE PARÁMETRO  \
1             ALC_TOT                             Alcalinidad Total   
5            CLOROF_A                                   Clorofila A   
6            COLI_FEC                            Coliformes Fecales   
7            COLI_TOT                            Coliformes Totales   
8              E_COLI                              Escherichia coli   
9                 COT                        Carbono Orgánico Total   
10            COT_SOL                      Carbono Orgánico Soluble   
11            DBO_SOL         Demanda Bioquímica de Oxígeno Soluble   
12            DBO_TOT           Demanda Bioquímica de Oxígeno Total   
13            DQO_SOL            Demanda Química de Oxígeno Soluble   
14            DQO_TOT              Demanda Química de Oxígeno Total   
15              N_NH3                           Nitrógeno Amoniacal   
16              N_NO2                         Nitrógeno de Nitritos   
17    

In [14]:
subconjunto_alcalinidad = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Alcalinidad", case=False)]
print(subconjunto_alcalinidad)


  CLAVE PARÁMETRO   NOMBRE PARÁMETRO UNIDAD DE MEDIDA
1         ALC_TOT  Alcalinidad Total       mg CaCO3/L


In [15]:
# Subconjuntos por categoría temática
subconjunto_alcalinidad = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Alcalinidad", case=False)]

subconjunto_bacteriologico = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Coliformes|Escherichia|Enterococo", case=False)]

subconjunto_oxigeno = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Oxígeno|DBO|DQO", case=False)]

subconjunto_organico = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Carbono|Orgánico", case=False)]

subconjunto_nitrogeno = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Nitrógeno|Amoniacal|Kjeldahl", case=False)]

subconjunto_toxicidad = simbologia_filtrada[simbologia_filtrada['NOMBRE PARÁMETRO'].str.contains("Toxicidad|Vibrio|Daphnia", case=False)]


In [16]:
# Extraer listas de columnas
parametros_alcalinidad = subconjunto_alcalinidad['CLAVE PARÁMETRO'].tolist()
parametros_bacteriologico = subconjunto_bacteriologico['CLAVE PARÁMETRO'].tolist()
parametros_oxigeno = subconjunto_oxigeno['CLAVE PARÁMETRO'].tolist()
parametros_organico = subconjunto_organico['CLAVE PARÁMETRO'].tolist()
parametros_nitrogeno = subconjunto_nitrogeno['CLAVE PARÁMETRO'].tolist()
parametros_toxicidad = subconjunto_toxicidad['CLAVE PARÁMETRO'].tolist()


In [17]:
# Seleccionar columnas de contexto común
columnas_contexto = ['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD']

# Crear DataFrames por categoría
df_alcalinidad = df_merged[columnas_contexto + parametros_alcalinidad]
df_bacteriologico = df_merged[columnas_contexto + parametros_bacteriologico]
df_oxigeno = df_merged[columnas_contexto + parametros_oxigeno]
df_organico = df_merged[columnas_contexto + parametros_organico]
df_nitrogeno = df_merged[columnas_contexto + parametros_nitrogeno]
df_toxicidad = df_merged[columnas_contexto + parametros_toxicidad]


In [21]:
# Recargar los DataFrames necesarios desde archivos previos
df_resultados = pd.read_csv("resultados_sonora.csv")
df_sitios = pd.read_csv("sitios_sonora.csv")

# Merge para reconstruir df_merged
df_merged = df_resultados.merge(df_sitios, on='CLAVE SITIO', how='left')

# Convertir fecha a datetime
df_merged['FECHA REALIZACIÓN'] = pd.to_datetime(df_merged['FECHA REALIZACIÓN'], dayfirst=True, errors='coerce')

# Crear resumen actualizado
df_alcalinidad = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_alcalinidad]
df_bacteriologico = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_bacteriologico]
df_oxigeno = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_oxigeno]
df_organico = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_organico]
df_nitrogeno = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_nitrogeno]
df_toxicidad = df_merged[['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD'] + parametros_toxicidad]

# Crear resumen
resumen = {
    'Categoría': ['Alcalinidad', 'Bacteriológico', 'Oxígeno', 'Orgánico', 'Nitrógeno', 'Toxicidad'],
    'Nº Filas': [
        df_alcalinidad.shape[0],
        df_bacteriologico.shape[0],
        df_oxigeno.shape[0],
        df_organico.shape[0],
        df_nitrogeno.shape[0],
        df_toxicidad.shape[0]
    ],
    'Nº Columnas': [
        df_alcalinidad.shape[1],
        df_bacteriologico.shape[1],
        df_oxigeno.shape[1],
        df_organico.shape[1],
        df_nitrogeno.shape[1],
        df_toxicidad.shape[1]
    ]
}

resumen_df = pd.DataFrame(resumen)
print(resumen_df)


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\2996506561.py:2: DtypeWarning: Columns (38,45,46,61,62,63,74,75,76,77,78,79,80,81,82,83,84,85,86,87,91,93,108,110,113,116,117,121,143,209,223,224,226,228,229,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resultados = pd.read_csv("resultados_sonora.csv")
C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\2996506561.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_merged['FECHA REALIZACIÓN'] = pd.to_datetime(df_merged['FECHA REALIZACIÓN'], dayfirst=True, errors='coerce')


        Categoría  Nº Filas  Nº Columnas
0     Alcalinidad      5901            7
1  Bacteriológico      5901           10
2         Oxígeno      5901           14
3        Orgánico      5901            9
4       Nitrógeno      5901           12
5       Toxicidad      5901           13


In [22]:
import plotly.express as px
import plotly.graph_objects as go

def graficar_parametros_plotly(df, parametros, categoria, agrupar_por='AÑO'):
    # Asegurar que la fecha esté en datetime y extraer el año si no existe
    if 'AÑO' not in df.columns:
        df['AÑO'] = df['FECHA REALIZACIÓN'].dt.year

    for param in parametros:
        if param in df.columns:
            df_filtrado = df[[param, agrupar_por]].dropna()

            # Histograma interactivo
            fig_hist = px.histogram(df_filtrado, x=param, nbins=30, title=f"{param} - Histograma ({categoria})")
            fig_hist.update_layout(bargap=0.1)
            fig_hist.show()

            # Boxplot por agrupador (año o municipio)
            fig_box = px.box(df_filtrado, x=agrupar_por, y=param, title=f"{param} - Boxplot por {agrupar_por}")
            fig_box.update_layout(xaxis_tickangle=-45)
            fig_box.show()


In [23]:
graficar_parametros_plotly(df_nitrogeno, parametros_nitrogeno, "Nitrógeno", agrupar_por='AÑO')


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\1086156253.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AÑO'] = df['FECHA REALIZACIÓN'].dt.year


In [27]:
# Revisar si algún parámetro contiene valores no numéricos (por ejemplo, strings tipo '<25')
# Hacer una limpieza antes de calcular cuantiles para evitar errores

def detectar_outliers_iqr_limpio(df, parametros, nombre_categoria):
    df_alertas = []

    for param in parametros:
        if param in df.columns:
            # Forzar a numérico, convertir errores a NaN
            df[param] = pd.to_numeric(df[param], errors='coerce')
            valores = df[param].dropna()

            if len(valores) > 0:
                q1 = valores.quantile(0.25)
                q3 = valores.quantile(0.75)
                iqr = q3 - q1
                lim_inf = q1 - 1.5 * iqr
                lim_sup = q3 + 1.5 * iqr

                # Crear columna de alerta
                df[f'ALERTA_{param}'] = df[param].apply(
                    lambda x: 1 if pd.notnull(x) and (x < lim_inf or x > lim_sup) else 0
                )

                # Guardar solo filas con alerta
                df_alertas_param = df[df[f'ALERTA_{param}'] == 1][
                    ['CLAVE SITIO', 'NOMBRE DEL SITIO_x', 'FECHA REALIZACIÓN', 'MUNICIPIO', 'LATITUD', 'LONGITUD', param]
                ].copy()
                df_alertas_param['PARÁMETRO'] = param
                df_alertas_param['CATEGORÍA'] = nombre_categoria
                df_alertas_param['VALOR'] = df_alertas_param[param]
                df_alertas_param.drop(columns=[param], inplace=True)

                df_alertas.append(df_alertas_param)

    if df_alertas:
        return pd.concat(df_alertas, ignore_index=True)
    else:
        return pd.DataFrame()

# Aplicar nuevamente a todos los subconjuntos
alertas_alcalinidad = detectar_outliers_iqr_limpio(df_alcalinidad, parametros_alcalinidad, "Alcalinidad")
alertas_bacteriologico = detectar_outliers_iqr_limpio(df_bacteriologico, parametros_bacteriologico, "Bacteriológico")
alertas_oxigeno = detectar_outliers_iqr_limpio(df_oxigeno, parametros_oxigeno, "Oxígeno")
alertas_organico = detectar_outliers_iqr_limpio(df_organico, parametros_organico, "Orgánico")
alertas_nitrogeno = detectar_outliers_iqr_limpio(df_nitrogeno, parametros_nitrogeno, "Nitrógeno")
alertas_toxicidad = detectar_outliers_iqr_limpio(df_toxicidad, parametros_toxicidad, "Toxicidad")

# Concatenar todas las alertas
alertas_total = pd.concat([
    alertas_alcalinidad,
    alertas_bacteriologico,
    alertas_oxigeno,
    alertas_organico,
    alertas_nitrogeno,
    alertas_toxicidad
], ignore_index=True)

from IPython.display import display

# Mostrar las primeras filas del DataFrame de alertas
display(alertas_total)

# O si prefieres un resumen tabular:
print("Total de alertas detectadas:", len(alertas_total))
print("Parámetros únicos detectados:", alertas_total['PARÁMETRO'].nunique())
print("Municipios afectados:", alertas_total['MUNICIPIO'].nunique())


C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\2676233388.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\2676233388.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Alan\AppData\Local\Temp\ipykernel_10104\2676233388.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,CLAVE SITIO,NOMBRE DEL SITIO_x,FECHA REALIZACIÓN,MUNICIPIO,LATITUD,LONGITUD,PARÁMETRO,CATEGORÍA,VALOR
0,OCNOR3986,AGUA PRIETA,2015-05-22,AGUA PRIETA,31.23720,-109.55489,ALC_TOT,Alcalinidad,490.2800
1,OCNOR3986,AGUA PRIETA,2016-11-06,AGUA PRIETA,31.23720,-109.55489,ALC_TOT,Alcalinidad,533.4000
2,OCNOR3986,AGUA PRIETA,2018-03-09,AGUA PRIETA,31.23720,-109.55489,ALC_TOT,Alcalinidad,483.8400
3,OCNOR4003,POZO TUBUTAMA,2015-07-02,TUBUTAMA,30.88404,-111.46740,ALC_TOT,Alcalinidad,585.8800
4,OCNOR4003,POZO TUBUTAMA,2018-12-20,TUBUTAMA,30.88404,-111.46740,ALC_TOT,Alcalinidad,466.5200
...,...,...,...,...,...,...,...,...,...
5889,OCNOR4106,PROVEEDORA DE SERVICIOS DEL PARQUE INDUSTRIAL ...,2017-10-19,NAVOJOA,26.99014,-109.43083,TOX_V_30_UT,Toxicidad,27.5400
5890,OCNOR4107,"CERVECERIA CUAUHTEMOC MOCTEZUMA, S.A. DE C.V. ...",2013-01-10,NAVOJOA,27.02460,-109.43370,TOX_V_30_UT,Toxicidad,55.8659
5891,OCNOR4107,"CERVECERIA CUAUHTEMOC MOCTEZUMA, S.A. DE C.V. ...",2014-06-26,NAVOJOA,27.02460,-109.43370,TOX_V_30_UT,Toxicidad,37.5900
5892,OCNOR4107,"CERVECERIA CUAUHTEMOC MOCTEZUMA, S.A. DE C.V. ...",2015-06-18,NAVOJOA,27.02460,-109.43370,TOX_V_30_UT,Toxicidad,25.5100


Total de alertas detectadas: 5894
Parámetros únicos detectados: 24
Municipios afectados: 43


In [28]:
alertas_total.groupby("MUNICIPIO").size().sort_values(ascending=False).head(10)


MUNICIPIO
NAVOJOA                   1728
CAJEME                    1084
HUATABAMPO                 886
GUAYMAS                    336
SAN IGNACIO RÍO MUERTO     292
AGUA PRIETA                214
NACOZARI DE GARCÍA         200
SAN LUIS RÍO COLORADO      163
CANANEA                    163
BÁCUM                      158
dtype: int64

In [30]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

# Centrar el mapa en Sonora
mapa = folium.Map(location=[29.0, -110.0], zoom_start=6)

# Crear clúster para agrupar puntos cercanos
marker_cluster = MarkerCluster().add_to(mapa)

# Colores por categoría
colores = {
    'Alcalinidad': 'blue',
    'Bacteriológico': 'red',
    'Oxígeno': 'green',
    'Orgánico': 'purple',
    'Nitrógeno': 'orange',
    'Toxicidad': 'black'
}

# Agregar marcadores
for _, fila in alertas_total.iterrows():
    categoria = fila['CATEGORÍA']
    color = colores.get(categoria, 'gray')
    tooltip = (
        f"<b>Parámetro:</b> {fila['PARÁMETRO']}<br>"
        f"<b>Valor:</b> {fila['VALOR']}<br>"
        f"<b>Fecha:</b> {fila['FECHA REALIZACIÓN']}<br>"
        f"<b>Municipio:</b> {fila['MUNICIPIO']}"
    )

    folium.CircleMarker(
        location=[fila['LATITUD'], fila['LONGITUD']],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=tooltip,
        tooltip=tooltip
    ).add_to(marker_cluster)

# Guardar el mapa como HTML
mapa.save("mapa_alertas_sonora.html")
